# OpenPages QuikBot

## Helper Functions

In [2]:
import textwrap
from IPython.display import display, HTML, Markdown

# displays new lines with line breaks
def print_df(df):

  display(df.style.set_properties(**{
      'text-align': 'left',
      'white-space': 'pre-wrap',
  }))

  HTML(df.to_html().replace("\\n","\<br\>"))

In [3]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# access colab secrets
import os
from google.colab import userdata

# load environment variables
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Install Libraries

In [6]:
# Required Packages
!pip install chromadb
!pip install langchain
!pip install pymupdf
!pip install sentence-transformers

!pip install trulens-eval

In [7]:
# import required libraries
import os
import re
import time
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from typing import List, Optional

# import libraries
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

# import LangChain libraries
from langchain import hub
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import PDFMinerLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import UnstructuredPDFLoader
from sentence_transformers.cross_encoder import CrossEncoder
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import LongContextReorder
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableParallel
from langchain.docstore.document import Document as LangchainDocument

In [8]:
# Set to display entire row content
pd.set_option('display.max_colwidth', None)

## Declare Constants

In [9]:
# path constants
ROOT_PATH = "/content/drive/MyDrive/ai-ml-projects/hf-llm-rag"
DATA_DIR = ROOT_PATH + "/data"
PERSIST_DIR = ROOT_PATH + "/store"
TEST_DIR = ROOT_PATH + "/test"
EVAL_FILE_PATH = TEST_DIR+"/eval_questions.txt"

In [10]:
# model names
HF_MODEL = "huggingfaceh4/zephyr-7b-beta"
EMBEDDING_MODEL = "BAAI/bge-large-en-v1.5"
RERANKER_MODEL = "colbert-ir/colbertv2.0"

## Load Models

In [11]:
# instantiate LLM
llm = HuggingFaceHub(
    repo_id=HF_MODEL,
    model_kwargs={
        "max_new_tokens": 512,
        "repetition_penalty": 1.1,
        "temperature": 0.2,
        "do_sample": True,
        "return_full_text": False
    }
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [12]:
# instantiate embedding model
def get_embedding_model():

  # check if GPU is available
  import tensorflow as tf
  device = 'cuda' if tf.test.gpu_device_name() else 'cpu'
  print('device:', device)

  hf_bge_embeddings = HuggingFaceBgeEmbeddings(
      model_name=EMBEDDING_MODEL,
      model_kwargs={'device': device},
      encode_kwargs={'normalize_embeddings': True} # set True to compute cosine similarity
  )

  # To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter.
  print(
      f"Model's maximum sequence length: {SentenceTransformer(EMBEDDING_MODEL).max_seq_length}"
  )

  return hf_bge_embeddings

In [13]:
# get embedding model
embedding_model = get_embedding_model()

device: cpu
Model's maximum sequence length: 512


# Retriever

## Load Data

In [14]:
# load documents into memory
def load_documents(dirPath):
  documents = []
  for root, _, files in os.walk(dirPath):
      for file in files:
          if file.endswith(".pdf"):
              print(f'Reading File: {file}')

              # read PDF
              loader = PyMuPDFLoader(os.path.join(root, file))
              document = loader.load()

              # append to docs
              documents += document
  return documents

In [15]:
# split documents
def chunk_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", ""]
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

## Generate Embeddings

In [16]:
# generate embeddings
def generate_embeddings():

  # load documents
  documents = load_documents(DATA_DIR)

  # chunk documents to honor the context length
  chunked_documents = chunk_documents(
    SentenceTransformer(EMBEDDING_MODEL).max_seq_length,  # We choose a chunk size adapted to our model
    documents,
    tokenizer_name=EMBEDDING_MODEL,
)

  # save indexes to disk
  vector_store = Chroma.from_documents(
      documents=chunked_documents,
      embedding=get_embedding_model(),
      collection_metadata={"hnsw:space": "cosine"},
      persist_directory=PERSIST_DIR
  )

  # zip indexes (optional)
  # !zip -FS -r {PERSIST_DIR+".zip"} {PERSIST_DIR"}

  return vector_store

In [17]:
# load embeddings
def load_embeddings():
  # load indexes from disk
  vector_store = Chroma(persist_directory=PERSIST_DIR, embedding_function=get_embedding_model())

  return vector_store

## Retrieve Store

In [18]:
# retrieve vector store
def get_store():
  if [f for f in os.listdir(PERSIST_DIR) if not f.startswith('.')] == []:
    print("Chroma DB is empty. Generating indexes...")
    return generate_embeddings()
  else:
    print("Chroma DB is not empty. Loading existing indexes!")
    return load_embeddings()

In [19]:
# instantiate base retriever
def get_base_retriever(top_k=4):

  # get the vector store
  vector_store = get_store()

  base_retriever = vector_store.as_retriever(
      search_type='mmr',
      search_kwargs={'k': top_k}
  )

  return base_retriever

In [20]:
# get the base retriever
base_retriever = get_base_retriever()

Chroma DB is not empty. Loading existing indexes!
device: cpu
Model's maximum sequence length: 512


## Evaluate

In [21]:
# results dataframe
df_eval = pd.DataFrame()

In [22]:
# read eval questions
eval_questions = []
with open(EVAL_FILE_PATH, 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)
eval_questions

['What is FastMap?',
 'What is a Role Template?',
 'What is the purpose of Object Reset?',
 'What is the purpose of Reporting Periods?',
 'List the system variables used in Expressions.',
 'Provide the steps to configure Watson Assistant in OpenPages?',
 'List the user administration permissions that can be delegated.',
 'What is the difference between PRE and POST position in Triggers?',
 'What are the features of Operational Risk Management in OpenPages?',
 'What are the different administrator permissions that can be delegated to a user group administrator?',
 'What are the different access controls available for non-participants for a standard stage within a workflow?']

In [23]:
# evaluate rag chain
def evaluate_rag(chain_name, rag_chain):

  columns = ['Chain', 'Question', 'Response', 'Time', 'Context']
  df = pd.DataFrame(columns=columns)

  for question in eval_questions:

    start_time = time.time()
    response = rag_chain.invoke(question)
    end_time = time.time()

    row = {
        'Chain': chain_name,
        'Question': response['query'],
        'Response': response['result'],
        'Time': "{:.2f}".format(round(end_time - start_time, 2)),
        'Context': response['context']
    }

    df = pd.concat([df, pd.DataFrame.from_records([row])])

  return df

# Reader

In [24]:
# query = "What is the purpose of Object Reset in OpenPages?"
# query = "Can you help me understand about Role Templates in OpenPages?"
# query = "List all the system variables used in Expressions."
query = "Provide the features of Operational Risk Managmement module in OpenPages."
# query = "Help me understand about Regulatory Compliance solution that OpenPages offers."

You are an AI assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If the answer cannot be deduced from the context, just say that you don't know. Keep the answer short and concise.</s>

In [25]:
prompt_template = """
<|system|>
You are an AI assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>
{context}</s>
<|user|>
{query}</s>
<|assistant|>
"""
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template)

In [26]:
rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | chat_prompt_template
    | llm
    | StrOutputParser()
)

qa_w_prompt = RunnableParallel(
    {"context": base_retriever, "query": RunnablePassthrough()}
).assign(result=rag_chain)

In [27]:
qa_w_prompt.invoke(query)

{'context': [Document(page_content='Chapter 15. Using OpenPages Operational Risk\nManagement objects\nIBM OpenPages Operational Risk Management combines document and process management with a\nmonitoring and decision support system that enables organizations to analyze, manage, and mitigate risk\nin a simple and efficient manner.\nIBM OpenPages Operational Risk Management helps automate the process of measuring and monitoring\noperational risk. It combines all risk data, including risk and control self assessments, loss events,\nscenario analysis, external losses, and key risk indicators (KRI), into a single integrated solution.\nIBM OpenPages Operational Risk Management includes the following key features:\n• Loss Events, which include the following activities:\n– Tracking, assessing, and managing both internal and external events that could result in operational\nloss.\n– Managing multiple impact events and recoveries that are associated with operational losses.\n• Risk and Control S

In [28]:
df_eval = pd.concat([df_eval, evaluate_rag('qa_w_prompt', qa_w_prompt)], ignore_index=True)

In [29]:
print_df(df_eval[['Chain',	'Question',	'Response',	'Time']])

,Chain,Question,Response,Time
0,qa_w_prompt,What is FastMap?,"FastMap is a productivity tool that helps automate the process of importing and batch processing object data into IBM OpenPages with Watson. It uses a data load template (a Microsoft Excel workbook in .xlsx format) to capture data for import, validates the data during import, and then populates the repository with the new or updated records. FastMap supports the import of Process and Risk objects but does not support the import of File and Signature objects or the system Comment field. It can also be used to import and export questionnaire template objects and their associated child objects, as well as the standard fields for questionnaire assessment instances. The tool includes options for cleaning up FastMap import templates stored on the server and controlling the formatting of exported data. Overall, FastMap helps increase efficiency and reduce manual effort required for data entry and management in OpenPages with Watson.",0.66
1,qa_w_prompt,What is a Role Template?,"In IBM OpenPages with Watson solutions, a Role Template is a predefined set of permissions and access levels that determines the privileges granted to a user. It includes application permissions, Object ACLs (RWDA), and access to features and functions. There are several Role Templates delivered with the solutions, such as ""All Permissions"" and ""All Data - Limited Admin,"" and some solutions may include additional Role Templates with a subset of permissions. By default, these Role Templates provide administrative rights or limited admin rights for all object types available for the solution. Administrators can also create custom Role Templates to define specific application permissions and security context points.",0.66
2,qa_w_prompt,What is the purpose of Object Reset?,"Object Reset is a feature in IBM OpenPages with Watson that allows administrators to automate the process of deleting, moving, renaming, copying, disassociating, or reassigning primary parents of objects based on specific criteria defined in a ruleset. This feature helps to streamline data management processes, reduce manual effort, and ensure consistency and accuracy of data across the application. It can also be used to prepare data for reporting purposes or to clean up obsolete or redundant data.",4.58
3,qa_w_prompt,What is the purpose of Reporting Periods?,"Reporting periods in IBM OpenPages with Watson are ""snapshots"" of the current state of the repository, typically created at the end of a quarter or year during documentation phases. They allow users to view and report on data from past periods without being affected by changes made to the repository. This feature helps organizations maintain historical records and facilitates auditing and compliance activities. Additionally, object resets, which are rule-based operations contained in a ruleset, can be used to automatically modify objects within the repository during reporting periods. Overall, reporting periods provide a way to manage and organize data over time in IBM OpenPages with Watson.",0.69
4,qa_w_prompt,List the system variables used in Expressions.,1. $APPLICATION_URL$ - A URL for OpenPages. 2. $COGNOS_URL$ - A URL for IBM Cognos Analytics. 3. $ApplicationText/application text key$ - Application text content. 4. $TASK_VIEW_URL$ - A URL to an object task view. 5. $System Fields:Task View URL$ - Deprecated. Use [$TASK_VIEW_URL$]. 6. $Setting/OpenPages/...$ - A registry setting value. 7. $END_USER$ - The user name of the signed on user. 8. $TODAY$ - Today's date. 9. $DaysFromNow/field group:field name$ - A day count from today to a given date.,0.69
5,qa_w_prompt,Provide the steps to configure Watson Assistant in OpenPages?,"1. In OpenPages, go to Administration > Integrations > Watson Assistant. 2. Click on ""Add Watson Assistant"" and enter your Watson Assistant credentials. 3. Select the desired Watson Assistant service and click ""Save"". 4. Configure the integr

In [30]:
query

'Provide the features of Operational Risk Managmement module in OpenPages.'

In [43]:
from operator import itemgetter
test_qa_chain = (
{
"context": itemgetter("query") | base_retriever,
"query": itemgetter("query")
}
| RunnableParallel({
"response": chat_prompt_template | llm,
"context": itemgetter("context"),
})
)

In [44]:
test_qa_chain.invoke(que)

TypeError: string indices must be integers